# Equation of state (EOS)
For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/common/module.html#workgraph_collections.common.eos.eos_workgraph).

## Visualizing the WorkGraph Builder




In [1]:
from workgraph_collections.common.eos import eos_workgraph
from aiida import load_profile
load_profile()

task = eos_workgraph.TaskCls()
task.to_html()


## Visualizing the WorkGraph

In [2]:
from workgraph_collections.common.eos import eos_workgraph
from aiida import load_profile
load_profile()

wg = eos_workgraph()
wg.to_html()

## Example: Quantum ESPRESSO
### Prepare the inputs and submit the workflow


In [3]:
from aiida import load_profile, orm
from ase.build import bulk
from workgraph_collections.common.eos import eos_workgraph
from aiida_quantumespresso.calculations.pw import PwCalculation

load_profile()

structure = orm.StructureData(ase=bulk("Si"))
code = orm.load_code("qe-7.2-pw@localhost")
parameters = {
    "CONTROL": {
        "calculation": "scf",
    },
    "SYSTEM": {
        "ecutwfc": 30,
        "ecutrho": 240,
        "occupations": "smearing",
        "smearing": "gaussian",
        "degauss": 0.1,
    },
}
# Load the pseudopotential family.
pseudo_family = orm.load_group("SSSP/1.3/PBEsol/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}

# ===============================================================================
scf_inputs = {
    "code": code,
    "parameters": orm.Dict(parameters),
    "kpoints": kpoints,
    "pseudos": pseudos,
    "metadata": metadata,
}
wg = eos_workgraph(
    structure=structure,
    scales=[0.98, 0.99, 1.0, 1.01, 1.02],
    scf_inputs=scf_inputs,
    calculator=PwCalculation,
)
wg.name = "QE-EOS-Si"
#------------------------- Submit the calculation -------------------
wg.submit(wait=True, timeout=300)


WorkGraph process created, PK: 54748


<WorkChainNode: uuid: 7c17abf8-237d-4f47-98ad-3b54b77e5f36 (pk: 54748) (aiida_workgraph.engine.workgraph.WorkGraphEngine)>

## Print the results

In [7]:
#------------------------- Print the output -------------------------
data = wg.tasks["fit_eos"].outputs["result"].value.get_dict()
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))



Result: 
B: 83.930472220007
v0: 40.947349586558
e0: -308.18973212512
v0: 40.947349586558


## CP2K


In [ ]:
from aiida import load_profile, orm
from ase.build import bulk
from workgraph_collections.common.eos import eos_workgraph
from aiida_cp2k.calculations import Cp2kCalculation


load_profile()

atoms = bulk("Si")
structure = orm.StructureData(ase=atoms)
code = orm.load_code("cp2k-ssmp-2024.2@localhost")
# Parameters.
parameters = {
    "GLOBAL": {
        "RUN_TYPE": "ENERGY_FORCE",
    },
    "FORCE_EVAL": {
        "METHOD": "Quickstep",
        "DFT": {
            "BASIS_SET_FILE_NAME": "BASIS_MOLOPT",
            "POTENTIAL_FILE_NAME": "POTENTIALS",
            "SCF": {
                "ADDED_MOS": 10,
                "SMEAR": {
                    "METHOD": "FERMI_DIRAC",
                    "ELECTRONIC_TEMPERATURE": 500,
                },
            },
            "KPOINTS": {
                "SCHEME": "MONKHORST-PACK 5 5 5",
            },
            "QS": {
                "EPS_DEFAULT": 1.0e-12,
                "METHOD": "GPW",
            },
            "MGRID": {
                "NGRIDS": 4,
                "CUTOFF": 500,
                "REL_CUTOFF": 50,
            },
            "XC": {
                "XC_FUNCTIONAL": {
                    "_": "PBE",
                },
            },
        },
        "SUBSYS": {
            "KIND": [
                {
                    "_": "Si",
                    "BASIS_SET": "DZVP-MOLOPT-GTH",
                    "POTENTIAL": "GTH-PBE",
                },
            ],
        },
    },
}
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}
basis_pseudo_files = {
    "basis": orm.load_node(39027),
    "pseudo": orm.load_node(39028),
}
scf_inputs = {
    "code": code,
    "parameters": parameters,
    "file": basis_pseudo_files,
    "metadata": metadata,
}
# ===============================================================================
wg = eos_workgraph(
    structure=structure,
    scales=[0.98, 0.99, 1.0, 1.01, 1.02],
    scf_inputs=scf_inputs,
    calculator=Cp2kCalculation,
)
wg.name = "CP2K-EOS-Si"
# print("correction_energies", correction_energies)
wg.submit()
